In [7]:
!pip install duckdb sqlalchemy psycopg2-binary pandas pyarrow boto3
!pip install sqlalchemy psycopg2-binary pandas pyarrow boto3
!pip install psycopg2-binary
!pip install dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
!pip install redshift_connector

   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 3.8/3.8 MB 42.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
!pip install duckdb pandas pyarrow



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import duckdb

# create memory databse(will not save data)
# conn = duckdb.connect(database=':memory:', read_only=False)

# create persistance database(data saved to local file mydatabase.duckdb)
conn = duckdb.connect(database='mydatabase.duckdb', read_only=False)



# Step2: Create Table
Assume we have a merged.csv data or something we could extract from previous step, and design the schema

In [2]:
create_table_query = """
DROP TABLE IF EXISTS stocks;
CREATE TABLE stocks (
    date DATE,
    s_symbol TEXT,
    s_open FLOAT,
    s_close FLOAT,
    c_symbol TEXT,
    c_open FLOAT,
    c_close FLOAT,
    sentiment FLOAT
)
"""
conn.execute(create_table_query)
print("Table 'merged_data' created successfully with the designed schema!")

Table 'merged_data' created successfully with the designed schema!


# Step 3: Load data from CSV file into the created table.

In [3]:

csv_file = 'test_data.csv'
copy_query = f"""
COPY stocks FROM '{csv_file}' (FORMAT CSV, HEADER TRUE);
"""
try:
    conn.execute(copy_query)
    print("Data loaded successfully into table 'merged_data'.")
except Exception as e:
    print("Error loading data:", e)


Data loaded successfully into table 'merged_data'.


# Step 4: Execute a sample SQL query.

In [4]:
# For example, select rows where sentiment > 0.
query = "SELECT * FROM stocks WHERE sentiment > 0"  # Removed trailing semicolon
try:
    result_df = conn.execute(query).fetchdf()
    print("Query executed successfully. Here are the first few rows:")
    print(result_df.head())
except Exception as e:
    print("Error executing query:", e)





Query executed successfully. Here are the first few rows:
        date s_symbol  s_open      s_close c_symbol   c_open  c_close  \
0 2024-01-01      SPY   370.0   375.000000      BTC  16000.0  16500.0   
1 2024-01-02     AAPL   150.5   152.300003      ETH   1100.0   1125.0   
2 2024-01-04     AMZN  3300.0  3320.000000      ETH   1120.0   1135.0   
3 2024-01-05     MSFT   310.0   315.500000      BTC  16200.0  16350.0   

   sentiment  
0       0.75  
1       0.55  
2       0.35  
3       0.65  


# Step 5: Export the query results to a Parquet file using DuckDB's COPY command.

In [5]:
parquet_file = 'positive_sentiment.parquet'
export_query = f"COPY ({query}) TO '{parquet_file}' (FORMAT PARQUET);"  # Now this works correctly
try:
    conn.execute(export_query)
    print(f"Query results exported successfully to Parquet file: {parquet_file}")
except Exception as e:
    print("Error exporting to Parquet file:", e)


Query results exported successfully to Parquet file: positive_sentiment.parquet


# Step 6: Upload the Parquet file to AWS S3 using boto3.
Credential Stored in AWS Secret Manager, and current user access has been grated

In [8]:
import boto3
import os
from dotenv import load_dotenv

# load .env
load_dotenv()

# Read credential
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
aws_region = os.getenv("AWS_REGION", "eu-north-1")  # default region

# ceate boto3 Session
session = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region
)

# access AWS Secrets Manager
secrets_client = session.client("secretsmanager")

# access S3 credentials
secret_name = "S3_access"
response = secrets_client.get_secret_value(SecretId=secret_name)
secret = response["SecretString"]

print("AWS Secrets Manager access successful!")


AWS Secrets Manager access successful!


In [9]:
import boto3
import json
from botocore.exceptions import ClientError

# Define AWS Secrets Manager details
SECRET_NAME = "S3_access"  # Name of your stored secret
REGION_NAME = "eu-north-1"  # The AWS region where Secrets Manager is configured

def get_aws_credentials():
    """Retrieve AWS credentials securely from AWS Secrets Manager."""
    try:
        # Create a Secrets Manager client
        session = boto3.session.Session()
        client = session.client(service_name='secretsmanager', region_name=REGION_NAME)

        # Retrieve the secret value
        get_secret_value_response = client.get_secret_value(SecretId=SECRET_NAME)
        secret = json.loads(get_secret_value_response['SecretString'])

        return secret["AWS_ACCESS_KEY_ID"], secret["AWS_SECRET_ACCESS_KEY"]
    
    except ClientError as e:
        print(f"Error retrieving secret: {e}")
        raise e  # Raise the exception for debugging

# Fetch credentials securely
aws_access_key_id, aws_secret_access_key = get_aws_credentials()

# Create S3 client using retrieved credentials
s3 = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=REGION_NAME
)

# Define S3 parameters
bucket_name = 'ucl-de'
s3_key = 'data/positive_sentiment.parquet'
parquet_file = 'positive_sentiment.parquet'

# Upload file to S3
try:
    s3.upload_file(parquet_file, bucket_name, s3_key)
    print(f"File uploaded successfully to S3: s3://{bucket_name}/{s3_key}")
except Exception as e:
    print("Error uploading file to S3:", e)


File uploaded successfully to S3: s3://ucl-de/data/positive_sentiment.parquet
